[![alt text](https://avatars1.githubusercontent.com/u/59831504?s=400&v=4 "MtheEPIC User Icon")](https://github.com/MtheEPIC/KaggleProj)

# Import libs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.axes_style("whitegrid")

import glob

from sklearn.decomposition import PCA

from sklearn.preprocessing import MinMaxScaler, binarize
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, plot_roc_curve, confusion_matrix, classification_report, precision_recall_curve, average_precision_score  
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, cross_val_predict

from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC

# Paths and Consts

In [38]:
r_hand_path = 'data\\HandRight.csv'
path = 'data\\Training\\*\\*'
start_delay = 7

# Funcs

In [210]:
def combine_hands(df, df2=None):
    if df2 is None:
        split1 = np.arange(0, df.shape[0], 2)
        split2 = np.arange(1, df.shape[0], 2)
        split_df1 = df.iloc[split1, :]
        split_df2 = df.iloc[split2, :]
        
        split_df1.set_index(np.arange(split_df1.shape[0]), inplace=True)
        split_df2.set_index(np.arange(split_df2.shape[0]), inplace=True)
        
        if split_df1.shape[0] > split_df2.shape[0]:
            split_df1 = split_df1.iloc[np.arange(split_df2.shape[0]), :]
        else:
            split_df2 = split_df2.iloc[np.arange(split_df1.shape[0]), :]
        
        print('delta is ............ ', split_df1.shape[0] - split_df2.shape[0])
        
        
        if split_df1['Hand Type'][0] == 'left':
            split_df1 = split_df1.add_suffix(' Left')
            split_df2 = split_df2.add_suffix(' Right')
        else:
            split_df1 = df1.add_suffix(' Right')
            split_df2 = split_df2.add_suffix(' Left')
        df = pd.concat([split_df1, split_df2], axis=1, sort=False)
#         print(df.head())
    else:
        df.set_index(np.arange(df.shape[0]), inplace=True)
        df2.set_index(np.arange(df2.shape[0]), inplace=True)
        df = df.add_suffix(' Left')
        df2 = df2.add_suffix(' Right')
        if df.shape[0] > df2.shape[0]:
            df = df.iloc[np.arange(df2.shape[0]), :]
        else:
            df2 = df2.iloc[np.arange(df.shape[0]), :]
        df = pd.concat([df, df2], axis=1, sort=False)
        
#     print(df.isnull().sum())
    return df

In [193]:
def mod_df(df, df2=None):
    df.rename(str.strip, axis='columns', inplace=True)
#     df.rename(str.replace(" ", "-"), axis='columns', inplace=True)
    df.drop('# hands', axis=1, inplace=True)
#     df = df.replace(['left', 'right'], [0, 1])
    df = df[df['Time']>=df['Time'][0]+start_delay]
    
    
    
    df = combine_hands(df, df2)
    
    start_time = df.iloc[0, 0]
    end_time = df.iloc[len(df)-1, 0]
    total_time = (end_time - start_time)
    start_frame = df.iloc[0, 1]
    end_frame = df.iloc[len(df)-1, 1]
    total_frame = (end_frame - start_frame)
    
    tmp1 = np.arange(0, df.shape[0], int(total_time))
    tmp2 = np.arange(1, df.shape[0], int(total_time))
    df['tmp'] = np.arange(df.shape[0])
    df_cut = df[df['tmp'].isin(np.append(tmp1, tmp2))]
    df_cut.drop('tmp', axis=1, inplace=True)
    
    
#     jump = total_frame//100
#     print(start_time, ' ', end_time, ' ', jump, ' ', start_frame, ' ', end_frame)
    
#     print(df_cut.head())
    return df_cut

In [3]:
# tmp = mod_df(train_spontan)
# # train_spontan.iloc[0, 0]
# tmp.info()
# tmp.head(22)

In [77]:
# train_sync = pd.read_csv('data/Training/Evyatar Cohen/Evyatar636771050902934708Sync.csv')
# train_sync.drop(' # hands', axis=1, inplace=True)
# train_sync = train_sync.replace(['left', 'right'], [0, 1])
# print(train_sync.columns)
# train_sync.head()

In [75]:
# train_sync.info()
# train_sync.describe()

In [78]:
# # After big join

# from sklearn import preprocessing
# mm_scaler = preprocessing.MinMaxScaler()
# X_train_minmax = mm_scaler.fit_transform(train_sync)
# tmp = mm_scaler.transform(train_sync)
# tmpdf = pd.DataFrame(tmp)
# # tmpdf = tmpdf.rename(columns=(np.arange(1, 22)))
# for i in range(tmpdf.shape[1]):
#     tmpdf.rename(columns={i: train_sync.columns[i]}, inplace=True)
# tmpdf

## data mod

In [5]:
# train_spontan.iloc[0, 0]

In [6]:
# train_spontan['Time']

In [7]:
# from os import listdir
# from os.path import isfile, join

# mypath = os.path.dirname(os.path.realpath(__file__))
# print(mypath)
# # onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [211]:
cnt = 0

train_df = pd.DataFrame()
r_hand_df = pd.read_csv(r_hand_path)
r_hand_df.dropna(inplace=True)
r_hand_df = r_hand_df[r_hand_df['Time']>=r_hand_df['Time'][0]+start_delay]
r_hand_df['value'] = 0
print(r_hand_df.shape)

for name in glob.glob(path):
    cnt = cnt +1
    print(name)
    df = pd.read_csv(name)
    print('origen', df.shape[0])
    
    df.dropna(inplace=True)
    if 'Alone' in name[-11:]: #add r hand
        df = mod_df(df, r_hand_df)
        df['value'] = 0
    else:
        df  = mod_df(df)
        if 'Spontan' in name[-11:]:
            df['value'] = 1
        elif 'Sync' in name[-11:]:
            df['value'] = 2
        else:
            print('ERORR')
    print('after', df.shape)
    train_df = train_df.append(df, ignore_index=True)
#     print(df.head())
cnt

(4007, 23)
data\Training\Evyatar Cohen\Evyatar636771050011691836Spontan.csv
origen 9054
delta is ............  0
after (124, 43)
data\Training\Evyatar Cohen\Evyatar636771050902934708Sync.csv
origen 9054
delta is ............  0
after (124, 44)
data\Training\Evyatar Cohen\Evyatar636771051823980826Alone.csv
origen 4476
after (120, 45)
data\Training\Evyatar Cohen\Evyatar636771052727603804Spontan.csv
origen 9054
delta is ............  0
after (124, 44)
data\Training\Evyatar Cohen\Evyatar636771053639929594Sync.csv
origen 9052
delta is ............  0
after (124, 44)
data\Training\Evyatar Cohen\Evyatar636771054555711409Alone.csv
origen 4526
after (122, 45)
data\Training\Nofar Social_Nuero\Nofar636759792183061005Spontan.csv
origen 9054
delta is ............  0
after (124, 44)
data\Training\Nofar Social_Nuero\Nofar636759793193137655Alone.csv
origen 4496
after (122, 45)
data\Training\Nofar Social_Nuero\Nofar636759794200223541Sync.csv
origen 9052
delta is ............  0
after (124, 43)
data\Tra

47

In [212]:
train_df

,Time Left,Frame ID Left,Hand Type Left,Position X Left,Position Y Left,Position Z Left,Velocity X Left,Velocity Y Left,Velocity Z Left,Pitch Left,...,Wrist Pos X Right,Wrist Pos Y Right,Wrist Pos Z Right,Elbow pos X Right,Elbow Pos Y Right,Elbow Pos Z Right,Grab Strenth Right,Grab Angle Right,Pinch Strength Right,value Right
0,71.41422,98073.0,left,-109.80610,228.3820,87.32578,-41.538020,116.70710,-60.09788,2.147859,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,71.43108,98075.0,left,-110.42900,230.8011,86.41932,-23.842480,125.97130,-58.77163,2.129599,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,72.52414,98201.0,left,-121.23690,276.2515,28.99516,79.100020,-246.42300,-81.58604,1.528455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,72.54060,98203.0,left,-120.68220,273.0325,27.80134,39.513900,-201.09340,-62.64017,1.515737,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73.63486,98329.0,left,-117.32820,222.7078,67.27760,169.424200,-337.23340,-171.53120,2.658031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531,850.96530,66978.0,left,-69.70062,197.6128,-17.89728,-26.862780,17.65697,24.74907,1.155033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5532,853.16490,67107.0,left,-60.93401,161.7846,-16.56654,42.082980,-63.10300,33.41759,0.766982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5533,853.19740,67109.0,left,-60.44543,160.9973,-15.86087,4.998864,-19.96488,23.01340,0.742324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5534,855.39620,67252.0,left,-66.33916,162.3058,-14.66839,2.070095,-92.90261,-69.09605,0.453430,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
# train_df['Hand Type Left'].unique()

array(['left', nan, 'right'], dtype=object)

In [89]:
train_df.nunique()

Time              8851
Frame ID          8573
Hand Type            2
Position X        8908
Position Y        8900
Position Z        8912
Velocity X        8917
Velocity Y        8916
Velocity Z        8918
Pitch             8908
Roll              8914
Yaw               8913
Wrist Pos X       8910
Wrist Pos Y       8897
Wrist Pos Z       8914
Elbow pos X       8912
Elbow Pos Y       8917
Elbow Pos Z       8911
Grab Strenth      1165
Grab Angle        6575
Pinch Strength    1645
value                3
dtype: int64

In [9]:
tmp = tmp.set_index(np.arange(0, tmp.shape[0]*2, 2))
tmp

NameError: name 'tmp' is not defined

In [ ]:
# r_hand_df = pd.read_csv(r_hand_path)
# rrrr = r_hand_df
r_hand_df

In [ ]:
r_hand_df = r_hand_df.set_index(np.arange(1, r_hand_df.shape[0]*2, 2))
r_hand_df

In [ ]:
r_hand_df.join(tmp, how='outer')
r_hand_df

In [94]:
train_sync

,Time,Frame ID,Hand Type,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,153.5306,107537,1,77.94024,254.7147,-7.674313,-0.480022,-59.95172,127.348100,0.875267,...,-0.332276,101.0865,198.40520,27.67238,248.6154,17.15779,115.17400,0.000000,0.000000,0.0
1,153.5306,107537,0,-103.68820,221.1642,3.092919,-2.858919,-180.74000,55.116670,1.245015,...,0.866406,-133.3421,158.32050,14.84969,-268.8730,-49.14569,44.19444,0.000000,0.000000,0.0
2,153.5476,107539,1,78.28185,253.5882,-5.193746,24.038670,-51.37317,143.180800,0.876265,...,-0.332286,101.4179,197.18370,29.99536,250.0403,15.59908,114.90930,0.000000,0.000000,0.0
3,153.5476,107539,0,-103.54540,218.4241,4.356668,10.185880,-126.31100,68.195580,1.248875,...,0.884464,-133.7134,155.77330,15.84225,-269.5829,-51.50067,44.97860,0.000000,0.000000,0.0
4,153.5644,107541,1,78.85633,252.8531,-2.370064,43.635580,-31.83131,169.023400,0.880479,...,-0.325577,101.7099,196.20520,32.58978,251.4731,14.51754,115.24890,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9049,228.4785,116172,0,-185.42340,103.9304,84.759130,251.999800,-176.13750,-60.528570,1.813329,...,1.803099,-237.9112,59.39917,70.98890,-380.6812,-142.62990,116.97360,0.057295,0.985836,0.0
9050,228.4947,116174,1,142.59520,182.1849,0.388569,-305.760400,-127.89070,7.957732,0.468154,...,-0.781808,196.0458,155.40500,38.88380,403.0817,55.84055,141.47960,0.000000,0.000000,0.0
9051,228.4947,116174,0,-181.42170,100.9335,84.579920,235.406100,-152.85790,-1.337266,1.806294,...,1.839208,-229.8396,52.18437,70.01520,-367.6695,-152.67630,118.36690,0.023611,0.969089,0.0
9052,228.5113,116176,1,136.98750,179.8985,1.137675,-273.629900,-104.38440,79.911710,0.463552,...,-0.750208,189.0327,151.86130,40.60957,399.8426,50.73706,133.44600,0.000000,0.000000,0.0


In [135]:
train_sync = pd.read_csv('data/Training/Evyatar Cohen/Evyatar636771050902934708Sync.csv')
# train_sync = train_sync.astype({' Frame ID': 'object'}).dtypes
# train_sync = train_sync.groupby(' Frame ID').agg(lambda x: pd.concat([x], axis=1, sort=False))
# train_sync.set_index(' Frame ID', inplace=True)
# pd.concat([], axis=1, sort=False))
split1 = np.arange(0, train_sync.shape[0], 2)
split2 = np.arange(1, train_sync.shape[0], 2)
df1 = train_sync.iloc[split1, :]
df2 = train_sync.iloc[split2, :]
df1.set_index(np.arange(df1.shape[0]), inplace=True)
df2.set_index(np.arange(df2.shape[0]), inplace=True)
if df1[' Hand Type'][0] == 'left':
    df1.add_suffix(' Left')
    df2.add_suffix(' Right')
else:
    df1 = df1.add_suffix(' Right')
    df2.add_suffix(' Left')
train_sync
df3 = pd.concat([df1, df2], axis=1, sort=False)
df3

,Time Right,Frame ID Right,Hand Type Right,# hands Right,Position X Right,Position Y Right,Position Z Right,Velocity X Right,Velocity Y Right,Velocity Z Right,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,153.5306,107537,right,2,77.94024,254.7147,-7.674313,-0.480022,-59.95172,127.348100,...,0.866406,-133.3421,158.32050,14.84969,-268.8730,-49.14569,44.19444,0.000000,0.000000,0.0
1,153.5476,107539,right,2,78.28185,253.5882,-5.193746,24.038670,-51.37317,143.180800,...,0.884464,-133.7134,155.77330,15.84225,-269.5829,-51.50067,44.97860,0.000000,0.000000,0.0
2,153.5644,107541,right,2,78.85633,252.8531,-2.370064,43.635580,-31.83131,169.023400,...,0.903786,-134.5467,153.86770,17.19151,-271.9559,-52.43427,46.00396,0.000000,0.000000,0.0
3,153.5803,107542,right,2,79.22472,252.5684,-0.942663,42.387210,-32.75925,164.239000,...,0.916129,-134.9740,152.38020,17.97288,-273.6234,-53.07392,46.90184,0.000000,0.000000,0.0
4,153.5968,107544,right,2,80.40151,252.2684,2.100434,72.927730,-27.86742,177.481700,...,0.940533,-136.0092,149.92730,19.93104,-277.5446,-53.55210,48.84118,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4522,228.4457,116168,right,2,156.35600,188.4865,1.395727,-204.240900,-132.66200,-28.802800,...,1.798288,-248.9223,68.04777,72.06062,-395.1380,-134.39580,103.38780,0.000000,0.907439,0.0
4523,228.4616,116170,right,2,152.57930,186.5244,0.870999,-224.656800,-111.57710,-27.992710,...,1.790117,-244.6177,64.94874,72.12418,-389.0879,-137.21240,112.00070,0.035171,0.928718,0.0
4524,228.4785,116172,right,2,148.02080,184.4683,0.423167,-280.636600,-127.04130,-22.315710,...,1.803099,-237.9112,59.39917,70.98890,-380.6812,-142.62990,116.97360,0.057295,0.985836,0.0
4525,228.4947,116174,right,2,142.59520,182.1849,0.388569,-305.760400,-127.89070,7.957732,...,1.839208,-229.8396,52.18437,70.01520,-367.6695,-152.67630,118.36690,0.023611,0.969089,0.0
